In [8]:
import json

import pandas as pd
import numpy as np

import redis
from redis.commands.search.field import NumericField, TagField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query, NumericFilter
from redis.commands.search.aggregation import AggregateRequest
from redis.commands.search import Search
import redis.commands.search.reducers as reducers

In [2]:
REDIS_HOST = '127.0.0.1'
REDIS_PORT = 6379
REDIS_PASSWORD = ''

conn = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD, encoding='utf-8', decode_responses=True)

# Tabla HASH

In [3]:
res = conn.hset('ventas:1', mapping={'region': 'Norte', 'categoria': 'Bebidas', 'producto': 'Jugos', 'venta': 500.50})
res = conn.hset('ventas:2', mapping={'region': 'Norte', 'categoria': 'Bebidas', 'producto': 'Lacteos', 'venta': 400.30})
res = conn.hset('ventas:3', mapping={'region': 'Norte', 'categoria': 'Bebidas', 'producto': 'Agua', 'venta': 230.20})
res = conn.hset('ventas:4', mapping={'region': 'Norte', 'categoria': 'Alimentos', 'producto': 'Frituras', 'venta': 1050.00})
res = conn.hset('ventas:5', mapping={'region': 'Norte', 'categoria': 'Alimentos', 'producto': 'Quesos', 'venta': 750.00})

res = conn.hset('ventas:6', mapping={'region': 'Sur', 'categoria': 'Bebidas', 'producto': 'Jugos', 'venta': 380.25})
res = conn.hset('ventas:7', mapping={'region': 'Sur', 'categoria': 'Bebidas', 'producto': 'Lacteos', 'venta': 350.00})
res = conn.hset('ventas:8', mapping={'region': 'Sur', 'categoria': 'Bebidas', 'producto': 'Agua', 'venta': 115.00})
res = conn.hset('ventas:9', mapping={'region': 'Sur', 'categoria': 'Alimentos', 'producto': 'Frituras', 'venta': 1300.00})
res = conn.hset('ventas:10', mapping={'region': 'Sur', 'categoria': 'Alimentos', 'producto': 'Quesos', 'venta': 600.00})

res = conn.hset('ventas:11', mapping={'region': 'Oriente', 'categoria': 'Bebidas', 'producto': 'Jugos', 'venta': 750.00})
res = conn.hset('ventas:12', mapping={'region': 'Oriente', 'categoria': 'Bebidas', 'producto': 'Lacteos', 'venta': 815.75})
res = conn.hset('ventas:13', mapping={'region': 'Oriente', 'categoria': 'Bebidas', 'producto': 'Agua', 'venta': 645.15})
res = conn.hset('ventas:14', mapping={'region': 'Oriente', 'categoria': 'Alimentos', 'producto': 'Frituras', 'venta': 415.90})
res = conn.hset('ventas:15', mapping={'region': 'Oriente', 'categoria': 'Alimentos', 'producto': 'Quesos', 'venta': 1000.00})

# Esquema e Indice

In [6]:
#INDEX GENERAL
try:
    conn.ft("idx:ventas_idx").dropindex()

except: 
    pass

schema = (
    TagField("region", as_name="region"),
    TagField("categoria", as_name="categoria"),
    TagField("producto", as_name="producto"),
    NumericField("venta", as_name="venta")
)

index_definition = IndexDefinition(prefix=["ventas:"], index_type=IndexType.HASH)
index = conn.ft("idx:ventas_idx")
res = index.create_index(fields=schema, definition=index_definition)

print(res)

OK


# Agregaciones

In [19]:
#APPLY
search = Search(conn, index_name="idx:ventas_idx")

aggregate_request = AggregateRequest(query='@categoria:{Bebidas}') \
    .load('region', 'categoria', 'producto', 'venta').apply(venta_iva='@venta * 1.15')

res = search.aggregate(aggregate_request)
print(len(res.rows))
for row in res.rows:
    print(row)

9
['region', 'Sur', 'categoria', 'Bebidas', 'producto', 'Jugos', 'venta', '380.25', 'venta_iva', '437.2875']
['region', 'Norte', 'categoria', 'Bebidas', 'producto', 'Jugos', 'venta', '500.5', 'venta_iva', '575.575']
['region', 'Oriente', 'categoria', 'Bebidas', 'producto', 'Agua', 'venta', '645.15', 'venta_iva', '741.9225']
['region', 'Oriente', 'categoria', 'Bebidas', 'producto', 'Jugos', 'venta', '750', 'venta_iva', '862.5']
['region', 'Sur', 'categoria', 'Bebidas', 'producto', 'Agua', 'venta', '115', 'venta_iva', '132.25']
['region', 'Oriente', 'categoria', 'Bebidas', 'producto', 'Lacteos', 'venta', '815.75', 'venta_iva', '938.1125']
['region', 'Norte', 'categoria', 'Bebidas', 'producto', 'Agua', 'venta', '230.2', 'venta_iva', '264.73']
['region', 'Norte', 'categoria', 'Bebidas', 'producto', 'Lacteos', 'venta', '400.3', 'venta_iva', '460.345']
['region', 'Sur', 'categoria', 'Bebidas', 'producto', 'Lacteos', 'venta', '350', 'venta_iva', '402.5']


In [27]:
#AGRUPAR
search = Search(conn, index_name="idx:ventas_idx")

aggregate_request = AggregateRequest(query='*').load('region', 'categoria', 'venta').group_by(['@region', '@categoria'], reducers.sum('@venta').alias('suma_ventas'))

res = search.aggregate(aggregate_request)
print(len(res.rows))
for row in res.rows:
    print(row)

6
['region', 'Oriente', 'categoria', 'Alimentos', 'suma_ventas', '1415.9']
['region', 'Norte', 'categoria', 'Bebidas', 'suma_ventas', '1131']
['region', 'Norte', 'categoria', 'Alimentos', 'suma_ventas', '1800']
['region', 'Oriente', 'categoria', 'Bebidas', 'suma_ventas', '2210.9']
['region', 'Sur', 'categoria', 'Alimentos', 'suma_ventas', '1900']
['region', 'Sur', 'categoria', 'Bebidas', 'suma_ventas', '845.25']


# Transacciones

In [34]:
#EXECUTE
res = conn.set('name', 'Mario')
print(res)

res = conn.get('name')
print(res)

flujo = conn.pipeline(transaction=True)
res = flujo.set('name', 'Luigi')

res = conn.get('name')
print(res)

flujo.execute()

res = conn.get('name')
print(res)

True
Mario
Mario
Luigi


In [35]:
#EXECUTE
res = conn.set('name', 'Mario')
print(res)

res = conn.get('name')
print(res)

flujo = conn.pipeline(transaction=True)
res = flujo.set('name', 'Luigi')

res = conn.get('name')
print(res)

flujo.discard()

res = conn.get('name')
print(res)

True
Mario
Mario
Mario
